# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [2]:
setprecision(BigFloat, 64)
Pkg.activate("..")
using DoubleFloats, BootstrapVirasoro

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [ ]:
# c = CentralCharge(β=1/(big"0.8" + big"0.1"*im))
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 20.

indices = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))

fields = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(fields..., Δmax=Δmax)

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1//2:1//2:50 for s in -10+1//r:1//r:10 if r*s%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:50]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:50 for s in -10+1//r:1//r:10],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:50 for s in -10+1//r:1//r:10],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    )
)

LoopSpectra = Dict(
    model => Spectrum(LoopFields[model], Δmax, interchiral=false)
    for model in (:On, :PSUn, :Potts)
)

SOn = LoopSpectra[:On];
SOn

Degenerate:
<1, 1>
<1, 3>
<1, 5>
<1, 7>
Non-diagonal:
(1, 0), (1, 1)
(2, -1//2), (2, 0), (2, 1//2), (2, 1)
(3, -2//3), (3, -1//3), (3, 0), (3, 1//3), (3, 2//3), (3, 1)

## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [11]:
Schan_On = ChannelSpectra(
    co, LoopSpectra[:On],
    signature = Dict(:s => 1, :t => 1//2, :u => 3//2)   
)
b = BootstrapSystem(Schan_On);
compute_linear_system!(b)
solve!(b, precision_factor=5)
b.consts

Channel s
Fields         | Structure constants          | Errors                     
--------------------------------------------------------------------
<1, 1>         |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
<1, 3>         |  3.72032e-02 - 3.74640e-01im |  5.12197e-01 + 0.00000e+00im
<1, 5>         |  3.45387e-01 + 1.56964e-02im |  4.96650e-01 + 0.00000e+00im
<1, 7>         |  1.69441e-01 + 4.34862e-02im |  5.41786e-01 + 0.00000e+00im
V_{(1, 0)}     | -3.05104e-02 + 1.59927e+00im |  2.20395e-01 + 0.00000e+00im
V_{(1, 1)}     |  1.96437e-02 + 2.09434e-01im |  2.72705e-01 + 0.00000e+00im
V_{(2, -1//2)} | -8.73479e-02 + 2.36294e-02im |  5.79182e-01 + 0.00000e+00im
V_{(2, 0)}     | -5.39634e-01 + 3.37303e-02im |  4.51939e-01 + 0.00000e+00im
V_{(2, 1//2)}  | -7.79261e-02 + 4.22799e-02im |  3.66798e-01 + 0.00000e+00im
V_{(2, 1)}     |  3.83829e-03 - 4.05964e-03im |  5.83105e-01 + 0.00000e+00im
V_{(3, -2//3)} |  1.93942e-01 + 2.80230e-01im |  5.25918e-01 + 0.00000e+00im

In [7]:
b.consts.errors

Dict{Symbol, Dict{Field{Complex{BigFloat}}, Complex{BigFloat}}} with 3 entries:
  :s => Dict(V_{(2, 1)}=>0.0+0.0im, <1, 1>=>0.0+0.0im, <1, 7>=>0.0+0.0im, V_{(2…
  :u => Dict(V_{(2, 1)}=>0.0+0.0im, <1, 1>=>0.0+0.0im, <1, 7>=>0.0+0.0im, V_{(2…
  :t => Dict(V_{(2, 1)}=>0.0+0.0im, <1, 1>=>0.0+0.0im, <1, 7>=>0.0+0.0im, V_{(2…